In [ ]:
# import gcsfs
import sys
import zarr
from pathlib import Path
import spikeinterface.full as si
import time
import numpy as np

sys.path.append("..")

from utils import get_median_and_lsb

%matplotlib notebook

## SAVE

In [ ]:
token = "/home/alessio/.config/gcloud/legacy_credentials/alessiop.buccino@gmail.com/adc.json"

In [ ]:
# fs = gcsfs.GCSFileSystem(project="neural-dynamics-338018", token=token)
# zarr_path = fs.get_mapper('aind-transfer-service-test/zarr-test-folder/si-toy-jobs8.zarr')

In [ ]:
storage_options={"token": token}

In [ ]:
np_version = 2

if np_version == 2:
    oe_folder = Path("/home/alessio/Documents/data/allen/npix-open-ephys/595262_2022-02-21_15-18-07")
else:
    oe_folder = Path("/home/alessio/Documents/data/allen/npix-open-ephys/618382_2022-03-31_14-27-03")
rec_oe = si.read_openephys(oe_folder, stream_id="0")
rec_oe = si.split_recording(rec_oe)[0]
print(rec_oe)

dtype = rec_oe.get_dtype()

# lsb_value, median_values = get_median_and_lsb(rec_oe)

In [ ]:
# # median correction
rec_to_compress = si.scale(rec_oe, gain=1., offset=-median_values, dtype=dtype)
rec_to_compress = si.scale(rec_oe, gain=1. / lsb_value, dtype=dtype)
rec_to_compress.set_channel_gains(rec_to_compress.get_channel_gains() * lsb_value)

In [ ]:
duration = rec_to_compress.get_num_samples() / rec_to_compress.get_sampling_frequency()

In [ ]:
zarr_path = f"gcs://aind-transfer-service-test/zarr-test-folder/{oe_folder.name}_compressed.zarr"

In [ ]:
# z = zarr.open(zarr_path, mode="w", storage_options=storage_options)

**NOTE**: it doesn't seem to work with `n_jobs>1`. Need to investigate this!

In [ ]:
n_jobs = 1

In [ ]:
t_start = time.perf_counter()
rec_gcloud = rec_to_compress.save(format="zarr", zarr_path=zarr_path, storage_options=storage_options,
                                  progress_bar=True, chunk_duration="1s", n_jobs=n_jobs)
t_stop = time.perf_counter()
elapsed_time = np.round(t_stop - t_start, 2)

xRT = duration / elapsed_time

print(f"Elapsed time to save to gcloud: {elapsed_time}s ({xRT} xRT)")

### Test on demand access

In [ ]:
rec_gcloud = si.read_zarr(zarr_path, storage_options=storage_options)

In [ ]:
rec_gcloud

In [ ]:
chunk_duration = [1, 2, 5, 10, 20, 40]
num_samples = rec_gcloud.get_num_samples()

for i, dur in enumerate(chunk_duration):
    print(f"Chunk duration {dur}s")
    samples = int(rec_gcloud.get_sampling_frequency() * dur)
    start_frame = np.random.randint(num_samples - samples - 1)
    end_frame = start_frame + samples

    t_start = time.perf_counter()
    tr_ch = rec_gcloud.get_traces(start_frame=start_frame, end_frame=end_frame)
    t_stop = time.perf_counter()
    elapsed_time = np.round(t_stop - t_start, 2)
    print(f"Elapsed time to retrieve {dur}s traces: {elapsed_time} s")

In [ ]:
rec_gcloud.get_annotation("compression_ratio")